In [4]:
import os
import pandas as pd
import numpy as np
import xarray as xr
from functools import reduce

import os
import pandas as pd
import numpy as np
import xarray as xr
from functools import reduce

def process_gnss_data_from_csv_robust(root_path: str, output_filename: str):
    """
    处理指定路径下的GNSS数据（CSV格式），整合为xarray.DataArray并输出为NetCDF文件。
    该版本对缺失数据更具鲁棒性，并增加了对不规整时间的重采样功能。

    Args:
        root_path (str): 包含各省份数据文件夹的根目录。
        output_filename (str): 输出的 .nc 文件名。
    """
    print(f"开始处理根目录: {root_path}")
    
    # 用于存储每个日期文件夹处理后的数据和站点列表
    all_folder_dataframes = []
    all_folder_stations = []

    # 1. 遍历根目录下的所有子文件夹
    for folder_name in os.listdir(root_path):
        folder_path = os.path.join(root_path, folder_name)
        if not os.path.isdir(folder_path):
            continue

        print(f"\n--- 正在处理文件夹: {folder_name} ---")
        
        csv_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.csv')]
        if not csv_files:
            print("  未找到 CSV 文件，跳过此文件夹。")
            continue
            
        print(f"  找到 {len(csv_files)} 个 CSV 文件。")

        daily_dfs = []
        stations_in_this_folder = set()

        for file in csv_files:
            try:
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                
                if not {'siteID', ' datetime', ' ztd'}.issubset(df.columns):
                    print(f"  文件 {file} 缺少必要的列，跳过。")
                    continue
                
                df['datetime'] = pd.to_datetime(df[' datetime'])
                df['siteID'] = df['siteID'].astype(str)
                df['ztd'] = pd.to_numeric(df[' ztd'], errors='coerce')
                
                daily_dfs.append(df)
                stations_in_this_folder.add(df['siteID'].iloc[0])

            except Exception as e:
                print(f"  读取或处理文件 {file} 时出错: {e}")
                continue
        
        if not daily_dfs:
            print("  未能成功读取任何文件，跳过此文件夹。")
            continue

        daily_combined_df = pd.concat(daily_dfs, ignore_index=True)
        print(f"  成功合并 {len(daily_combined_df)} 行数据，包含 {len(stations_in_this_folder)} 个站点。")
        
        all_folder_dataframes.append(daily_combined_df)
        all_folder_stations.append(stations_in_this_folder)

    if not all_folder_dataframes:
        print("\n处理完成，但未找到任何有效数据。程序退出。")
        return

    common_stations = reduce(lambda a, b: a.intersection(b), all_folder_stations)
    print(f"\n--- 数据整合 ---")
    print(f"在所有 {len(all_folder_stations)} 个文件夹中找到 {len(common_stations)} 个公共站点。")

    if not common_stations:
        print("所有文件夹之间没有公共站点，无法创建数据集。程序退出。")
        return

    print("正在合并所有文件夹的数据...")
    final_combined_df = pd.concat(all_folder_dataframes, ignore_index=True)

    print(f"过滤数据，只保留 {len(common_stations)} 个公共站点...")
    final_combined_df = final_combined_df[final_combined_df['siteID'].isin(common_stations)]
    final_combined_df.dropna(subset=['datetime', 'ztd'], inplace=True)

    if final_combined_df.empty:
        print("最终数据为空，可能是公共站点没有有效值。程序退出。")
        return

    # --- 从这里开始是关键修改 ---

    # 5. 【新增】时间重采样 (Resampling) 和聚合
    print("正在将时间戳重采样到规整的6分钟间隔...")
    # 使用 .dt.round('6T') 将时间四舍五入到最近的6分钟点。
    # '6T' 是 pandas 的频率字符串，代表 6 分钟。
    # 例如, '2025-05-06T07:11:38' 会被四舍五入到 '2025-05-06T07:12:00'
    final_combined_df['datetime'] = final_combined_df['datetime'].dt.round('6T')

    print("对同一时间间隔内的多个数据点进行聚合（计算平均值）...")
    # 由于多个原始时间点可能被归入同一个6分钟点，我们需要对它们进行聚合。
    # 按站点(siteID)和新的规整时间(datetime)分组，然后计算 ztd 的平均值。
    # .reset_index() 会将分组结果重新变回 DataFrame。
    final_aggregated_df = final_combined_df.groupby(['datetime', 'siteID'])['ztd'].mean().reset_index()

    # 6. 【修改】将聚合后的数据转换为二维表格 (time x station)
    print("正在将聚合后的数据转换为 time x station 的二维格式...")
    # 注意：这里要使用我们刚刚创建的聚合后的 DataFrame: final_aggregated_df
    pivot_df = final_aggregated_df.pivot_table(
        index='datetime', 
        columns='siteID', 
        values='ztd'
    )
    
    # --- 修改结束 ---

    # 对时间和站点进行排序，使数据更规整
    pivot_df = pivot_df.sort_index()
    pivot_df = pivot_df.reindex(sorted(pivot_df.columns), axis=1)

    # 7. 创建 xarray.DataArray
    print("正在创建 xarray.DataArray...")
    da = xr.DataArray(
        data=pivot_df.values.astype(np.float32),
        dims=("time", "station"),
        coords={
            "time": pivot_df.index.values,
            "station": pivot_df.columns.values.astype(str)
        },
        name="ztd"
    )
    
    da.attrs['long_name'] = 'Zenith Total Delay'
    da.attrs['units'] = 'mm'
    da.attrs['description'] = 'GNSS Zenith Total Delay data, resampled to 6-minute intervals. Missing values are filled with NaN.'

    # 8. 保存为 .nc 文件
    try:
        encoding = {
            'ztd': {'_FillValue': np.nan, 'dtype': 'float32'},
            'time': {'units': 'seconds since 1970-01-01 00:00:00', 'dtype': 'double'}
        }
        print(f"正在保存数据到: {output_filename}")
        da.to_netcdf(output_filename, encoding=encoding)
        print("\n处理成功！NetCDF 文件已生成。")
        print("最终数据维度信息:")
        print(da)
    except Exception as e:
        print(f"保存为 NetCDF 文件时出错: {e}")


if __name__ == '__main__':
    # --- 请修改以下参数 ---
    # 1. 您的数据根目录路径
    data_root_path = r'E:\huawei_gnss' 
    
    # 2. 您希望输出的 .nc 文件名
    output_nc_file = 'E:/gnss_ztd_combined_robust_6min.nc'
    # --- 参数修改结束 ---

    if not os.path.exists(data_root_path):
        print(f"错误：路径 '{data_root_path}' 不存在。请检查您的路径设置。")
    else:
        process_gnss_data_from_csv_robust(data_root_path, output_nc_file)

开始处理根目录: E:\huawei_gnss

--- 正在处理文件夹: zhejiang-20250514-6min ---
  找到 885 个 CSV 文件。
  成功合并 1506443 行数据，包含 885 个站点。

--- 正在处理文件夹: zhejiang-20250522-6min ---
  找到 1392 个 CSV 文件。
  成功合并 2258021 行数据，包含 1392 个站点。

--- 正在处理文件夹: zhejiang-20250530-6min ---
  找到 935 个 CSV 文件。
  成功合并 1598436 行数据，包含 935 个站点。

--- 正在处理文件夹: zhejiang-20250607-6min ---
  找到 1545 个 CSV 文件。
  成功合并 2394346 行数据，包含 1545 个站点。

--- 正在处理文件夹: zhejiang-20250616-6min ---
  找到 1545 个 CSV 文件。
  成功合并 2198632 行数据，包含 1545 个站点。

--- 正在处理文件夹: zhejiang-20250623-6min ---
  找到 1502 个 CSV 文件。
  成功合并 2177916 行数据，包含 1502 个站点。

--- 正在处理文件夹: zhejiang-20250629-6min ---
  找到 1542 个 CSV 文件。
  成功合并 2075315 行数据，包含 1542 个站点。

--- 正在处理文件夹: zhejiang-20250705-6min ---
  找到 1440 个 CSV 文件。
  成功合并 2038759 行数据，包含 1440 个站点。

--- 正在处理文件夹: zhejiang-20250711-6min ---
  找到 1440 个 CSV 文件。
  成功合并 2236752 行数据，包含 1440 个站点。

--- 正在处理文件夹: zhejiang-20250720-6min ---
  找到 1428 个 CSV 文件。
  成功合并 1947882 行数据，包含 1428 个站点。

--- 数据整合 ---
在所有 10 个文件夹中找到 566 个公共站点。
正在合并所有文件夹的